In [20]:
import os 
import pandas as pd
import numpy as np

from LoadData import LoadData
from FilterBasic import Filter
import scipy
from scipy.stats import skew
from scipy.stats import kurtosis

import functools
import operator

In [21]:
gap_animals = ["S7063", "S7064", "S7069"]
wt_animals = ["S7068", "S7070", "S7071"] 
gap_rec = [f for f in os.listdir() if "GAP.npy" in f]
wt_rec = [f for f in os.listdir() if "WT.npy" in f] 

In [22]:
# Here choose the animal recording and start/end
directory = '/Volumes/Macintosh HD/Users/gokceuzun/Desktop/4. SENE/Honors Project/'
filename = 'S7063_GAP.npy'
start = 15324481
end = 36959040

# Load the recording and slice
data = LoadData(directory=directory, filename=filename, start=start, end=end)
unfiltered_data = data.get_dat()
unfiltered_data = data.slice_data(unfiltered_data)

# Filter and reshape into epochs
fltr_instance = Filter(unfiltered_data)
filtered_data = fltr_instance.butter_bandpass()
reshaped_data = fltr_instance.reshape_filtered_data(filtered_data)
packet_loss_array, packet_loss_idx = fltr_instance.packet_loss_indices(reshaped_data)

### Calculate PSD of each epoch

In [23]:
d, f = calculate_psd(filtered_data, False) # PSD of the entire filtered data
d

,0,1,2,3,4,5,6,7,8,9,...,617,618,619,620,621,622,623,624,625,626
0,362430.766864,1.971925e+06,2.713468e+06,1.950672e+06,1.354437e+06,1.051602e+06,872560.953413,752679.446326,651004.311424,575673.016364,...,1.656097e-09,1.518197e-09,1.456829e-09,1.426460e-09,1.402453e-09,1.385608e-09,1.372371e-09,1.362939e-09,1.357337e-09,6.777361e-10
1,362320.209204,1.971213e+06,2.712872e+06,1.950653e+06,1.354434e+06,1.051416e+06,872404.863169,752650.287115,651003.321541,575631.544921,...,1.655712e-09,1.517849e-09,1.456478e-09,1.426119e-09,1.402125e-09,1.385279e-09,1.372044e-09,1.362613e-09,1.357012e-09,6.775738e-10
2,362001.220545,1.968971e+06,2.708611e+06,1.946813e+06,1.351599e+06,1.049518e+06,870876.970309,751206.340959,649729.960292,574512.351594,...,1.652371e-09,1.514754e-09,1.453481e-09,1.423183e-09,1.399240e-09,1.382425e-09,1.369216e-09,1.359804e-09,1.354214e-09,6.761768e-10
3,362090.627859,1.969996e+06,2.709932e+06,1.947695e+06,1.352283e+06,1.050031e+06,871271.159498,751628.234129,650189.404950,575070.851986,...,1.653109e-09,1.515453e-09,1.454248e-09,1.423925e-09,1.399968e-09,1.383159e-09,1.369948e-09,1.360535e-09,1.354945e-09,6.765420e-10
4,362020.515733,1.969445e+06,2.710043e+06,1.948285e+06,1.352692e+06,1.050242e+06,871446.998971,751759.296496,650233.222592,575018.818494,...,1.654355e-09,1.516595e-09,1.455325e-09,1.424985e-09,1.401004e-09,1.384176e-09,1.370952e-09,1.361530e-09,1.355934e-09,6.770355e-10
5,362141.618443,1.970187e+06,2.710855e+06,1.948932e+06,1.353392e+06,1.050817e+06,871910.535836,752025.843811,650431.773447,575153.125036,...,1.654450e-09,1.516709e-09,1.455390e-09,1.425029e-09,1.401059e-09,1.384224e-09,1.371000e-09,1.361576e-09,1.355980e-09,6.770583e-10
6,362433.103811,1.971773e+06,2.713329e+06,1.950668e+06,1.354423e+06,1.051600e+06,872503.791184,752665.216026,651010.577283,575661.881049,...,1.656169e-09,1.518290e-09,1.456938e-09,1.426549e-09,1.402557e-09,1.385705e-09,1.372468e-09,1.363035e-09,1.357433e-09,6.777838e-10
7,362261.572897,1.970416e+06,2.709814e+06,1.947045e+06,1.351742e+06,1.049851e+06,871197.905856,751375.428295,649637.296207,574382.975914,...,1.652814e-09,1.515149e-09,1.453924e-09,1.423602e-09,1.399642e-09,1.382827e-09,1.369616e-09,1.360202e-09,1.354611e-09,6.763751e-10
8,364201.558976,1.981330e+06,2.726226e+06,1.959630e+06,1.360589e+06,1.056442e+06,876603.132591,756188.909372,653998.487944,578244.289468,...,1.662848e-09,1.524489e-09,1.462778e-09,1.432299e-09,1.408190e-09,1.391262e-09,1.377970e-09,1.368496e-09,1.362869e-09,6.804980e-10
9,364580.424187,1.983483e+06,2.729653e+06,1.962460e+06,1.362705e+06,1.058105e+06,877909.862895,757275.971176,654969.253865,579212.098241,...,1.666071e-09,1.527378e-09,1.465656e-09,1.435092e-09,1.410949e-09,1.393998e-09,1.380682e-09,1.371193e-09,1.365557e-09,6.818404e-10


In [24]:
channels = ['ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8', 'ch9', 'ch10', 'ch11', 'ch12', 'ch13', 'ch14', 'ch15', 'ch16']
epochs = ["epoch" + str(i+1) for i in range(17280)]

In [25]:
df = pd.DataFrame.from_records(reshaped_data).T
df.columns = channels
df

,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16
0,"[14.517459411141314, 9.68202610913161, 9.37631...","[6.050170734109054, -2.0207160460076317, -13.7...","[9.63688947589803, 3.7142859877897805, -1.5093...","[16.864971058053964, 9.115626692360443, 5.6163...","[11.159300021033165, 6.424321460539171, 4.7179...","[8.129794494053032, 3.375708117759906, 3.78707...","[0.8473580547179342, 0.007191232725786811, 0.6...","[8.844793067805156, 4.037892191399959, 2.40289...","[13.181216879084008, 12.174484120039835, 10.29...","[9.431838352368654, 6.720866792806385, 3.86986...","[12.489661728481714, 7.484080742398889, -2.529...","[12.43047904931879, 3.319452440512472, -10.401...","[13.903798144624062, 6.87080645860974, -7.0397...","[16.700808484146297, 12.457772590411349, 2.940...","[5.349365644732483, -1.7334609100843918, -1.97...","[12.934711059559888, 9.00001607626519, 3.78429..."
1,"[36.4266790802226, 37.45161835525188, 36.29982...","[17.86926813303952, 10.832831376219836, 7.7981...","[17.373062464403958, 11.208706021879825, 4.759...","[19.368857307744587, 17.61401630303911, 13.627...","[15.657765799993287, 11.904772083306987, 7.934...","[3.931212761200225, 3.169270765225269, 0.96338...","[5.457504634287783, 3.3754144975655223, 0.9960...","[10.199214219449523, 8.591305421801986, 6.7654...","[19.809464391916222, 16.120034123821, 7.988776...","[12.973657905350242, 10.321479527024419, 1.962...","[12.423308377212972, 8.810642081665145, 2.3829...","[16.634932259569403, 10.080922395636092, 2.671...","[5.19427298882477, 4.201516153224029, 0.143936...","[-75.77322003201037, -81.75302162352028, -87.0...","[15.8066342418478, 8.606767405628048, 5.298414...","[6.845480844268209, 6.632760674044208, 1.92068..."
2,"[12.18718848007577, 12.418511515058585, 11.371...","[-2.9705696272221243, 7.212761223958914, 12.57...","[-8.482443964821652, -4.970348225200457, -2.76...","[-13.628134370476555, -9.327326853881704, -13....","[-0.3623393166640225, 3.017262757508239, 0.454...","[-6.054367122816045, -4.986288905589369, -5.33...","[3.971702121916909, 6.397293644777816, 7.82024...","[-10.992613392955578, -7.5363479576844075, -10...","[13.210989800391015, 17.61315812325442, 24.831...","[-13.447165090579508, -7.643289373531568, -0.6...","[16.19516514939972, 22.80997479624355, 28.9087...","[-5.899296868967301, 2.286931115255762, 7.7584...","[64.62876347871693, 71.1897023801094, 74.17420...","[47.75024909334192, 54.07229956698956, 59.3259...","[-2.355897118388694, 4.408866555097269, 17.776...","[-8.08796715456345, -6.917294175084411, -3.179..."
3,"[22.45868885159402, 26.375392629176368, 23.832...","[35.19874748429637, 25.15688647414171, 21.2061...","[16.806713043060682, 21.65107429174415, 17.286...","[11.52258381572238, 18.604994211082463, 16.006...","[18.711160134697217, 23.845868189236413, 21.91...","[11.74691086176005, 12.121683137451827, 10.093...","[20.47354979508988, 19.287697222610188, 16.298...","[20.14990211100938, 27.077086601137495, 25.325...","[-5.15292633496397, -1.1498515544579648, -4.26...","[4.014244471835262, 9.808018459871455, 5.50922...","[23.33501353142814, 31.862472654904675, 24.945...","[22.67005213796568, 27.20525553052873, 19.0430...","[12.19538418760882, 20.345319788688478, 12.351...","[32.11537582254555, 42.51703438483007, 36.8274...","[36.35912084543649, 28.34218709352538, 23.0620...","[6.373559548238479, 12.26601981476471, 8.92572..."
4,"[45.453638891809234, 43.81670438892779, 32.327...","[33.72922942124266, 33.22467792137823, 37.2872...","[80.40987155576158, 78.65233757551383, 70.2879...","[82.18952858916796, 83.19087822451888, 74.0666...","[76.67738813666313, 78.0648691848876, 68.55683...","[27.88888706462962, 25.8918433085489, 18.79166...","[22.830998780588935, 22.897828735725085, 18.84...","[92.30416034657131, 93.45628067489523, 84.8207...","[-9.329944815818173, -10.186481756488067, -12....","[26.087376845997646, 24.061222667330313, 22.15...","[52.29225996318509, 50.75793898648382, 46.0440...","[78.6885679393323

### PSD OF EACH EPOCH FOR EACH CHANNEL

In [26]:
df_PSD = df.applymap(lambda x: scipy.signal.welch(x, 250.4, window = 'hann', nperseg = 1252)[1])
df_PSD.columns = ['psd_ch1', 'psd_ch2', 'psd_ch3', 'psd_ch4', 'psd_ch5', 'psd_ch6', 'psd_ch7', 'psd_ch8', 'psd_ch9', 'psd_ch10', 'psd_ch11', 'psd_ch12', 'psd_ch13', 'psd_ch14', 'psd_ch15', 'psd_ch16']
df_PSD

,psd_ch1,psd_ch2,psd_ch3,psd_ch4,psd_ch5,psd_ch6,psd_ch7,psd_ch8,psd_ch9,psd_ch10,psd_ch11,psd_ch12,psd_ch13,psd_ch14,psd_ch15,psd_ch16
0,"[2.465627998034787, 359.24379863567304, 334.76...","[20.852408602975384, 151.6766030253765, 261.72...","[10.25770819617087, 49.75894519459817, 72.2769...","[0.8354863652244978, 2.675492560053667, 5.2761...","[5.549473179733447, 56.84230285108216, 103.459...","[7.413771303716582, 41.01484837718977, 63.7029...","[15.004053847889196, 70.68885753093281, 90.711...","[65.3182166105392, 784.2366295854388, 582.1368...","[204.0626987881347, 722.6963930509929, 394.018...","[0.5520327934884212, 40.567199022561134, 53.89...","[5.923851233692403, 47.86416009037123, 94.9622...","[8.89582324728388, 36.2018751671448, 24.438585...","[46.72368667414834, 259.7540434712161, 312.255...","[910.5880538071427, 2889.384203509909, 1551.83...","[15.35729705634952, 65.55292747111181, 76.8674...","[2.524916083958584, 23.46466498077941, 42.3150..."
1,"[386.30577624552615, 1043.248936812859, 107.00...","[47.929483310292476, 142.64866158943641, 47.94...","[0.12094293151420943, 18.323032347843334, 34.1...","[3.682958101155993, 28.690416943829913, 40.440...","[16.74473061202012, 76.73088514568724, 72.8736...","[8.69770901309434, 72.38295622685052, 108.3585...","[4.933198654099496, 18.66255136580681, 11.5849...","[14.697810942797547, 205.9480706609101, 239.09...","[82.28767083718532, 95.27194372198785, 34.6024...","[10.0400884168341, 34.33485543265659, 17.69703...","[18.383458211322925, 370.61673796355535, 39.85...","[3.803709561142188, 35.433360217856496, 63.518...","[12.580763929793065, 97.35806119370885, 18.963...","[767.651898280374, 19188.253618481336, 12419.0...","[49.5064972644561, 174.01360150297236, 138.662...","[0.6524949407533135, 13.618949954585483, 9.911..."
2,"[224.08596356842673, 725.508532479231, 138.631...","[1.458445397347766, 17.497262571750678, 58.223...","[142.60646824503968, 978.4042468361118, 909.86...","[38.95036399671516, 246.78163014149325, 204.79...","[1.1304366766254919, 0.7476525839305929, 7.216...","[3.0386655066676695, 11.560489909422587, 9.710...","[0.7792703369785569, 8.694026913572136, 14.594...","[58.59045039261001, 376.95509217412314, 265.06...","[1.8056335288815244, 460.887140727353, 188.545...","[3.930581414931163, 66.47366522118769, 162.754...","[935.6903584088542, 2497.643161209873, 1272.18...","[20.242108304436922, 235.4582405644948, 405.66...","[107.994194582453, 254.49924899764363, 934.499...","[112.77671544960751, 3786.1234083207214, 3714....","[1.8787454254521867, 19.92241304374953, 65.471...","[0.07080195790204898, 0.9417464848780995, 3.86..."
3,"[9.973353120030442, 28.2619963102494, 14.38229...","[3.339817761702652, 14.801599857970647, 12.212...","[0.3284044685821844, 75.5932501995836, 75.4522...","[5.08398572256648, 35.7227828002662, 45.537851...","[7.850055378117495, 53.332377188651456, 76.014...","[0.6216171727008639, 3.908380139941891, 5.0226...","[4.585471845722352, 6.558552133438308, 2.54425...","[664.625402737239, 1919.4855781264748, 957.015...","[809.4372626321455, 2556.0102889379455, 1171.6...","[14.720792807976574, 48.91865373624429, 24.490...","[24.924734725164093, 76.80043132370298, 55.681...","[5.995874234132535, 47.96631471699836, 41.9749...","[4.694390450277345, 80.55565588703351, 225.166...","[607.0840170688987, 2090.2499166713633, 1230.6...","[1.573460990690208, 8.071774626447917, 6.20321...","[12.515936377537486, 39.460930390932006, 32.32..."
4,"[100.39960273457491, 526.1345016532301, 227.63...","[0.5219397079838253, 4.269375063764799, 13.705...","[0.4086631098849882, 46.60535901139473, 15.988...","[15.599683743996795, 90.78275377428318, 19.256...","[0.006544580049229647, 18.833619592050677, 6.6...","[0.9104917364503021, 9.550176782469505, 25.047...","[0.5493073361536257, 9.023236856585955, 23.072...","[93.11015677098604, 258.7002491500516, 175.154...","[315.8967713987784, 945.5710521890604, 575.556...","[0.06105997075121417, 4.939782349996261, 3.256...","[15.5

### Total power

In [27]:
import scipy 
import numpy
from scipy.integrate import simps

def total_power(x):
    epoch_psd = scipy.signal.welch(x, 250.4, window = 'hann', nperseg = 1252)[1]
    freqs = scipy.signal.welch(x, 250.4, window = 'hann', nperseg = 1252)[0]
    freq_res = freqs[1] - freqs[0] # 0.2
    total_power = simps(epoch_psd, dx=freq_res)
    return total_power

In [59]:
df_totalP = df.applymap(lambda x: total_power(x))
df_totalP.columns = ['totalP_ch1', 'totalP_ch2', 'totalP_ch3', 'totalP_ch4', 'totalP_ch5', 'totalP_ch6', 'totalP_ch7', 'totalP_ch8', 'totalP_ch9', 'totalP_ch10', 'totalP_ch11', 'totalP_ch12', 'totalP_ch13', 'totalP_ch14', 'totalP_ch15', 'totalP_ch16']
df_totalP

,totalP_ch1,totalP_ch2,totalP_ch3,totalP_ch4,totalP_ch5,totalP_ch6,totalP_ch7,totalP_ch8,totalP_ch9,totalP_ch10,totalP_ch11,totalP_ch12,totalP_ch13,totalP_ch14,totalP_ch15,totalP_ch16
0,2.721804e+02,2.012054e+02,1.825586e+02,1.806714e+02,1.711144e+02,9.156068e+01,7.174101e+01,4.479009e+02,3.206327e+02,1.121376e+02,1.728005e+02,1.738668e+02,3.251317e+02,1.233250e+03,1.168723e+02,1.144978e+02
1,4.454980e+02,1.684707e+02,1.670174e+02,1.832186e+02,1.832526e+02,9.334461e+01,6.292561e+01,2.112866e+02,8.303136e+01,9.231648e+01,2.753379e+02,1.958325e+02,1.724827e+02,7.130952e+03,2.315905e+02,1.076374e+02
2,3.799666e+02,1.532916e+02,7.063393e+02,3.753419e+02,2.399599e+02,8.619874e+01,5.931148e+01,3.442615e+02,1.907571e+02,1.488301e+02,1.097938e+03,3.563936e+02,6.035577e+02,2.007909e+03,1.406271e+02,1.652527e+02
3,6.687876e+02,5.684963e+02,1.086375e+03,1.364891e+03,1.089420e+03,3.720022e+02,1.564027e+02,1.646430e+03,1.095036e+03,4.574626e+02,8.194373e+02,1.043604e+03,1.275727e+03,1.831856e+03,5.313329e+02,8.212786e+02
4,5.953847e+02,3.776555e+02,6.882901e+02,7.470899e+02,6.750165e+02,2.282750e+02,1.145656e+02,6.512359e+02,4.677259e+02,2.533632e+02,5.892758e+02,6.399146e+02,9.820339e+02,3.664794e+03,3.418816e+02,4.502018e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,8.980074e+07,9.003629e+07,8.985351e+07,8.979213e+07,8.974071e+07,8.977436e+07,8.985197e+07,9.012569e+07,9.051756e+07,9.036637e+07,8.987121e+07,9.006485e+07,8.969005e+07,9.029809e+07,9.043498e+07,9.052058e+07
17276,1.166017e+04,1.001800e+04,9.781511e+03,1.169177e+04,1.201681e+04,1.017000e+04,1.113457e+04,1.555771e+04,1.209595e+04,1.200254e+04,1.235164e+04,1.183132e+04,1.302568e+04,9.876389e+03,8.333108e+03,1.149674e+04
17277,1.483042e+02,1.554396e+02,3.374252e+02,8.982654e+02,2.753835e+02,2.188728e+02,5.833071e+01,2.554619e+03,6.445230e+02,2.293928e+02,2.028644e+02,2.819186e+02,3.442979e+02,3.168322e+03,1.173219e+03,1.992585e+02
17278,9.420314e+02,2.311865e+02,1.286817e+03,1.977708e+03,1.497456e+03,3.862927e+02,1.266765e+02,1.194538e+03,3.432478e+02,4.685271e+02,1.213398e+03,1.790506e+03,1.976784e+03,2.109000e+03,6.240794e+02,8.247291e+02


### 5-9Hz

In [29]:
def band_power(x, low, high):
    
    epoch_psd = scipy.signal.welch(x, 250.4, window = 'hann', nperseg = 1252)[1]
    freqs = scipy.signal.welch(x, 250.4, window = 'hann', nperseg = 1252)[0]

    # Find intersecting values in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)
    
    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Compute the absolute power by approximating the area under the curve
    band_power = simps(epoch_psd[idx_band], dx=freq_res)
    
    return band_power

In [30]:
df_band1 = df.applymap(lambda x: band_power(x, 5, 9))
df_band1.columns = ['5-9Hz_ch1', '5-9Hz_ch2', '5-9Hz_ch3', '5-9Hz_ch4', '5-9Hz_ch5', '5-9Hz_ch6', '5-9Hz_ch7', '5-9Hz_ch8', '5-9Hz_ch9', '5-9Hz_ch10', '5-9Hz_ch11', '5-9Hz_ch12', '5-9Hz_ch13', '5-9Hz_ch14', '5-9Hz_ch15', '5-9Hz_ch16']
df_band1

,5-9Hz_ch1,5-9Hz_ch2,5-9Hz_ch3,5-9Hz_ch4,5-9Hz_ch5,5-9Hz_ch6,5-9Hz_ch7,5-9Hz_ch8,5-9Hz_ch9,5-9Hz_ch10,5-9Hz_ch11,5-9Hz_ch12,5-9Hz_ch13,5-9Hz_ch14,5-9Hz_ch15,5-9Hz_ch16
0,2.343105e+01,5.868816e+00,1.822646e+01,2.959856e+01,1.820697e+01,1.760846e+01,5.051423e+00,2.259134e+01,4.636866e+00,2.130666e+01,2.937275e+01,2.497349e+01,3.291367e+01,1.886293e+01,6.799712e+00,3.031984e+01
1,2.871584e+01,2.410655e+01,2.279039e+01,3.166905e+01,2.375184e+01,1.900287e+01,1.030487e+01,2.524920e+01,7.698884e+00,2.272577e+01,3.270400e+01,2.757565e+01,2.625777e+01,1.960538e+01,2.718099e+01,3.603052e+01
2,2.888903e+01,2.589402e+01,4.153212e+01,4.877184e+01,4.810415e+01,1.667284e+01,1.300357e+01,3.646952e+01,7.766251e+00,1.898085e+01,3.350148e+01,3.564787e+01,3.243582e+01,3.829082e+01,2.532105e+01,2.780207e+01
3,8.939961e+01,1.036378e+02,1.789391e+02,2.337224e+02,1.638284e+02,4.314336e+01,1.429093e+01,1.350264e+02,7.866129e+00,3.754661e+01,1.076783e+02,1.765320e+02,2.227248e+02,2.065638e+02,9.215734e+01,1.197172e+02
4,1.020344e+02,9.042409e+01,1.526845e+02,1.660375e+02,1.595713e+02,5.491613e+01,1.979665e+01,1.279736e+02,2.004538e+01,5.761812e+01,1.202702e+02,1.373743e+02,1.576349e+02,1.460751e+02,7.826063e+01,1.128076e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,4.422589e+06,4.434325e+06,4.429120e+06,4.420830e+06,4.416641e+06,4.421720e+06,4.425951e+06,4.435372e+06,4.456590e+06,4.447519e+06,4.422714e+06,4.434587e+06,4.407453e+06,4.443256e+06,4.452891e+06,4.457574e+06
17276,2.726470e+02,1.275999e+01,3.149640e+02,5.046941e+02,3.569567e+02,1.330680e+02,4.078272e+01,3.778988e+02,4.155383e+01,1.867491e+02,3.286400e+02,3.776637e+02,4.830681e+02,3.764399e+02,4.876775e+01,3.379212e+02
17277,5.102353e+01,1.748992e+01,4.480651e+01,5.656022e+01,4.199763e+01,3.909744e+01,1.050356e+01,4.099482e+01,1.247026e+01,3.650324e+01,4.203566e+01,4.369738e+01,4.774119e+01,4.482000e+01,1.210963e+02,4.251150e+01
17278,3.193646e+02,2.573282e+01,3.933432e+02,5.925054e+02,4.307847e+02,1.470602e+02,4.223320e+01,4.212311e+02,5.044219e+01,2.319478e+02,4.132256e+02,4.635167e+02,5.831347e+02,4.756657e+02,3.994924e+01,3.831166e+02


### 1-20Hz

In [31]:
df_band2 = df.applymap(lambda x: band_power(x, 1, 20))
df_band2.columns = ['1-20Hz_ch1', '1-20Hz_ch2', '1-20Hz_ch3', '1-20Hz_ch4', '1-20Hz_ch5', '1-20Hz_ch6', '1-20Hz_ch7', '1-20Hz_ch8', '1-20Hz_ch9', '1-20Hz_ch10', '1-20Hz_ch11', '1-20Hz_ch12', '1-20Hz_ch13', '1-20Hz_ch14', '1-20Hz_ch15', '1-20Hz_ch16']
df_band2

,1-20Hz_ch1,1-20Hz_ch2,1-20Hz_ch3,1-20Hz_ch4,1-20Hz_ch5,1-20Hz_ch6,1-20Hz_ch7,1-20Hz_ch8,1-20Hz_ch9,1-20Hz_ch10,1-20Hz_ch11,1-20Hz_ch12,1-20Hz_ch13,1-20Hz_ch14,1-20Hz_ch15,1-20Hz_ch16
0,7.040782e+01,4.537190e+01,1.098551e+02,1.186399e+02,9.355278e+01,4.979546e+01,2.576674e+01,8.132080e+01,2.247844e+01,6.152333e+01,1.070918e+02,1.139031e+02,1.245356e+02,1.056082e+02,5.019320e+01,7.932153e+01
1,5.972776e+01,6.266293e+01,8.925690e+01,9.577248e+01,8.728604e+01,3.732982e+01,3.273937e+01,7.062131e+01,1.741629e+01,4.474721e+01,8.432421e+01,8.974683e+01,8.161994e+01,1.206868e+02,9.246792e+01,6.844014e+01
2,1.217535e+02,9.535479e+01,1.834846e+02,1.991389e+02,1.828842e+02,6.224621e+01,4.217127e+01,1.495499e+02,2.578517e+01,5.741226e+01,1.470516e+02,1.564405e+02,1.753043e+02,1.783005e+02,8.721332e+01,1.227695e+02
3,5.996498e+02,5.000187e+02,9.529845e+02,1.241910e+03,9.330183e+02,3.341642e+02,1.391155e+02,8.515919e+02,1.536247e+02,4.039403e+02,7.139567e+02,9.214719e+02,1.111023e+03,9.664164e+02,4.694721e+02,7.502626e+02
4,3.787406e+02,3.401718e+02,6.242294e+02,6.707225e+02,6.194846e+02,2.005684e+02,9.939771e+01,5.127693e+02,9.779237e+01,2.287018e+02,4.723238e+02,5.860283e+02,6.184170e+02,6.203063e+02,3.066392e+02,4.037865e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,5.727661e+07,5.742365e+07,5.738499e+07,5.727043e+07,5.721863e+07,5.724521e+07,5.728919e+07,5.743754e+07,5.769918e+07,5.759356e+07,5.729551e+07,5.744434e+07,5.708705e+07,5.757473e+07,5.768890e+07,5.772247e+07
17276,4.645112e+02,1.144136e+02,7.278033e+02,1.083024e+03,8.903105e+02,2.195378e+02,7.628731e+01,7.778779e+02,7.825494e+01,3.061880e+02,7.099028e+02,1.010233e+03,1.058208e+03,8.244387e+02,3.201349e+02,5.651191e+02
17277,1.082030e+02,6.382550e+01,1.175216e+02,1.612360e+02,1.078202e+02,7.603523e+01,2.830021e+01,1.178587e+02,2.898577e+01,7.476821e+01,1.222830e+02,1.195641e+02,1.298161e+02,1.139244e+02,7.064142e+02,9.688644e+01
17278,5.204154e+02,1.137232e+02,8.077436e+02,1.165566e+03,9.737444e+02,2.330658e+02,7.365890e+01,8.147781e+02,8.690553e+01,3.505122e+02,8.142021e+02,1.094597e+03,1.167939e+03,9.157257e+02,1.456330e+02,6.159655e+02


### 60-90Hz

In [32]:
df_band3 = df.applymap(lambda x: band_power(x, 60, 90))
df_band3.columns = ['60-90Hz_ch1', '60-90Hz_ch2', '60-90Hz_ch3', '60-90Hz_ch4', '60-90Hz_ch5', '60-90Hz_ch6', '60-90Hz_ch7', '60-90Hz_ch8', '60-90Hz_ch9', '60-90Hz_ch10', '60-90Hz_ch11', '60-90Hz_ch12', '60-90Hz_ch13', '60-90Hz_ch14', '60-90Hz_ch15', '60-90Hz_ch16']
df_band3

,60-90Hz_ch1,60-900Hz_ch2,60-90Hz_ch3,60-90Hz_ch4,60-90Hz_ch5,60-90Hz_ch6,60-90Hz_ch7,60-90Hz_ch8,60-90Hz_ch9,60-90Hz_ch10,60-90Hz_ch11,60-90Hz_ch12,60-90Hz_ch13,60-90Hz_ch14,60-90Hz_ch15,60-90Hz_ch16
0,1.343411,5.907273,3.172263,3.061775,3.313760,1.089151,0.709646,2.093917,1.168275,1.739103,2.257892,3.175335,3.053428,2.823513,4.337315,1.337070
1,1.171670,8.238356,2.745785,2.952294,2.898580,0.869773,1.096463,1.805185,1.520523,1.419400,2.288583,3.617299,2.749485,3.363625,8.896061,1.618385
2,1.145140,5.645872,2.230173,2.425457,2.413526,0.853598,0.743078,1.536387,1.064132,1.304620,1.767611,2.866621,2.311119,2.677715,3.496594,1.300651
3,0.875408,4.676427,1.836889,1.688715,1.820281,0.674909,0.368858,1.304059,0.976510,1.466609,1.847508,2.380323,1.777561,1.924560,2.517294,1.460860
4,1.110663,3.618568,1.649294,1.762692,1.754986,0.656110,0.376804,1.213144,0.848362,0.930216,1.356415,1.643581,1.410187,1.652077,3.062929,1.077645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,728790.412713,730582.312621,730173.166091,728680.001610,728200.941328,728679.376485,729048.691891,730772.353019,734178.374446,733072.822764,729120.827106,731041.178432,726641.750877,732605.630818,733218.573493,734741.698132
17276,3.207805,28.324563,10.221843,8.796183,9.889271,2.112728,1.857047,6.270533,3.273784,4.248865,8.200026,13.409572,9.876359,9.215929,16.581936,4.615624
17277,2.040978,13.766375,4.176318,3.981767,4.323161,1.963478,2.152706,3.398618,2.853666,2.861777,3.954944,5.014304,3.906351,4.694304,14.255504,2.255298
17278,3.411393,18.974553,9.575268,7.582331,9.754392,2.946192,3.146676,6.305490,3.027462,3.614981,7.595677,13.127592,7.970147,8.514860,12.221284,3.669976


### Centre of Mass

In [33]:
from CentreOfMass import CentreofMass

In [49]:
def calculate_centre_of_mass(x):
    psd = scipy.signal.welch(x, 250.4, window = 'hann', nperseg = 1252)[1]
    freqs = scipy.signal.welch(x, 250.4, window = 'hann', nperseg = 1252)[0]
    
    df_pow = pd.DataFrame(psd, columns=['Power'])
    df_pow['Frequency'] = freqs
    
    com = CentreofMass(df_pow, 0, 'ch1', 'S7063')
    pow_arr, freq_arr = com.frequency_slice()
    
    return com.calculate_cent_mass(pow_arr, freq_arr)

In [51]:
df_com = df.applymap(lambda x: calculate_centre_of_mass(x))
df_com.columns = ['com_ch1', 'com_ch2', 'com_ch3', 'com_ch4', 'com_ch5', 'com_ch6', 'com_ch7', 'com_ch8', 'com_ch9', 'com_ch10', 'com_ch11', 'com_ch12', 'com_ch13', 'com_ch14', 'com_ch15', 'com_ch16']
df_com

,com_ch1,com_ch2,com_ch3,com_ch4,com_ch5,com_ch6,com_ch7,com_ch8,com_ch9,com_ch10,com_ch11,com_ch12,com_ch13,com_ch14,com_ch15,com_ch16
0,9.331454,15.614040,10.720296,10.267741,10.812952,8.938585,9.007449,10.712621,10.171549,9.968187,10.309924,11.553498,10.114020,11.372047,14.386849,10.145598
1,10.757088,15.827216,13.238324,12.694538,13.358557,10.710965,11.321313,12.124277,14.198049,10.926133,11.385654,14.175166,12.810096,9.787968,12.769103,10.265652
2,8.596175,11.819183,9.216601,8.454209,9.501043,9.220411,9.860131,8.848687,12.568613,10.764400,8.829426,10.225134,8.949214,9.144682,11.559704,9.169602
3,6.559620,5.611436,6.837163,6.957995,6.931119,6.437004,5.977075,6.722738,5.890190,6.316071,7.021305,7.172321,7.133647,6.896171,5.406762,6.666093
4,7.216632,7.417016,7.784281,7.629725,7.853454,7.146947,6.617264,7.380930,7.274418,7.048826,7.549322,7.684448,7.566899,7.754646,7.345152,7.521053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,3.482890,3.482722,3.482848,3.482703,3.482713,3.483319,3.483441,3.482630,3.483269,3.482931,3.482575,3.482593,3.483040,3.482309,3.481298,3.483238
17276,14.140410,20.128966,16.073599,15.420546,16.684699,13.280118,14.581081,15.679961,15.038894,13.653569,16.363432,17.517268,16.183842,16.312299,10.644762,13.959724
17277,10.053812,15.855316,14.165550,11.652885,13.820882,9.460747,12.157867,11.515564,13.677781,10.551499,11.013464,13.491333,12.534347,14.663347,6.714213,10.556903
17278,13.396795,18.272026,15.307474,14.766871,15.889052,13.374448,15.257248,15.105353,15.064226,13.360480,15.693837,17.010102,15.487317,15.680597,14.060508,13.514744


### PEAKS

In [53]:
import numpy as np

In [54]:
def find_peak(x):
    psd = scipy.signal.welch(x, 250.4, window = 'hann', nperseg = 1252)[1]
    freqs = scipy.signal.welch(x, 250.4, window = 'hann', nperseg = 1252)[0]
    maxIndex = np.argmax(psd)
    return psd[maxIndex]

In [55]:
df_peak = df.applymap(lambda x: find_peak(x))
df_peak.columns = ['peak_ch1', 'peak_ch2', 'peak_ch3', 'peak_ch4', 'peak_ch5', 'peak_ch6', 'peak_ch7', 'peak_ch8', 'peak_ch9', 'peak_ch10', 'peak_ch11', 'peak_ch12', 'peak_ch13', 'peak_ch14', 'peak_ch15', 'peak_ch16']
df_peak

,peak_ch1,peak_ch2,peak_ch3,peak_ch4,peak_ch5,peak_ch6,peak_ch7,peak_ch8,peak_ch9,peak_ch10,peak_ch11,peak_ch12,peak_ch13,peak_ch14,peak_ch15,peak_ch16
0,3.592438e+02,2.617280e+02,7.227698e+01,7.055819e+01,1.034592e+02,6.370297e+01,9.071191e+01,7.842366e+02,7.226964e+02,5.389897e+01,9.496230e+01,3.620188e+01,3.122555e+02,2.889384e+03,7.686748e+01,4.231507e+01
1,1.043249e+03,1.426487e+02,7.697345e+01,9.784436e+01,7.805003e+01,1.083585e+02,3.588183e+01,2.390938e+02,9.527194e+01,6.226523e+01,3.706167e+02,1.180846e+02,9.735806e+01,1.918825e+04,1.740136e+02,6.958130e+01
2,7.255085e+02,5.822337e+01,9.784042e+02,2.467816e+02,6.524986e+01,2.539743e+01,1.694804e+01,3.769551e+02,4.608871e+02,1.627543e+02,2.497643e+03,4.056641e+02,9.344997e+02,3.786123e+03,6.547115e+01,7.242545e+01
3,4.164796e+02,3.409513e+02,6.845058e+02,8.284011e+02,6.968258e+02,2.453652e+02,1.092489e+02,1.919486e+03,2.556010e+03,3.229554e+02,5.920535e+02,6.889802e+02,6.789933e+02,2.090250e+03,3.184835e+02,5.230859e+02
4,5.261345e+02,1.491573e+02,3.066398e+02,2.681095e+02,2.470932e+02,9.594650e+01,5.085365e+01,2.587002e+02,9.455711e+02,1.153862e+02,1.783934e+02,2.732552e+02,9.680241e+02,7.037665e+03,1.306786e+02,1.385917e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,4.205781e+07,4.219054e+07,4.202020e+07,4.202144e+07,4.200313e+07,4.203898e+07,4.205961e+07,4.221591e+07,4.238937e+07,4.233780e+07,4.207070e+07,4.212223e+07,4.203639e+07,4.235317e+07,4.233113e+07,4.237381e+07
17276,3.732476e+04,3.331111e+04,2.921970e+04,3.483914e+04,3.604960e+04,3.343258e+04,3.706679e+04,4.911082e+04,4.097197e+04,3.881021e+04,3.777222e+04,3.485337e+04,3.794699e+04,2.936243e+04,2.710437e+04,3.618863e+04
17277,5.548019e+01,7.750845e+01,3.928228e+02,2.448370e+03,2.572501e+02,2.515039e+02,3.613355e+01,5.648120e+03,1.430513e+03,3.463768e+02,7.655238e+01,2.339479e+02,3.779507e+02,6.843339e+03,8.717514e+02,1.506231e+02
17278,6.329242e+02,7.410402e+01,7.400123e+02,1.139434e+03,8.194502e+02,2.529457e+02,7.737150e+01,8.192314e+02,5.030449e+02,4.132098e+02,7.647712e+02,8.592354e+02,1.081552e+03,2.077815e+03,1.348819e+03,6.813305e+02


## Feature Selection

In [62]:
df_features = pd.concat([df_PSD, df_com, df_totalP, df_band1, df_band2, df_band3, df_peak], axis=1)
df_features

,psd_ch1,psd_ch2,psd_ch3,psd_ch4,psd_ch5,psd_ch6,psd_ch7,psd_ch8,psd_ch9,psd_ch10,...,peak_ch7,peak_ch8,peak_ch9,peak_ch10,peak_ch11,peak_ch12,peak_ch13,peak_ch14,peak_ch15,peak_ch16
0,"[2.465627998034787, 359.24379863567304, 334.76...","[20.852408602975384, 151.6766030253765, 261.72...","[10.25770819617087, 49.75894519459817, 72.2769...","[0.8354863652244978, 2.675492560053667, 5.2761...","[5.549473179733447, 56.84230285108216, 103.459...","[7.413771303716582, 41.01484837718977, 63.7029...","[15.004053847889196, 70.68885753093281, 90.711...","[65.3182166105392, 784.2366295854388, 582.1368...","[204.0626987881347, 722.6963930509929, 394.018...","[0.5520327934884212, 40.567199022561134, 53.89...",...,9.071191e+01,7.842366e+02,7.226964e+02,5.389897e+01,9.496230e+01,3.620188e+01,3.122555e+02,2.889384e+03,7.686748e+01,4.231507e+01
1,"[386.30577624552615, 1043.248936812859, 107.00...","[47.929483310292476, 142.64866158943641, 47.94...","[0.12094293151420943, 18.323032347843334, 34.1...","[3.682958101155993, 28.690416943829913, 40.440...","[16.74473061202012, 76.73088514568724, 72.8736...","[8.69770901309434, 72.38295622685052, 108.3585...","[4.933198654099496, 18.66255136580681, 11.5849...","[14.697810942797547, 205.9480706609101, 239.09...","[82.28767083718532, 95.27194372198785, 34.6024...","[10.0400884168341, 34.33485543265659, 17.69703...",...,3.588183e+01,2.390938e+02,9.527194e+01,6.226523e+01,3.706167e+02,1.180846e+02,9.735806e+01,1.918825e+04,1.740136e+02,6.958130e+01
2,"[224.08596356842673, 725.508532479231, 138.631...","[1.458445397347766, 17.497262571750678, 58.223...","[142.60646824503968, 978.4042468361118, 909.86...","[38.95036399671516, 246.78163014149325, 204.79...","[1.1304366766254919, 0.7476525839305929, 7.216...","[3.0386655066676695, 11.560489909422587, 9.710...","[0.7792703369785569, 8.694026913572136, 14.594...","[58.59045039261001, 376.95509217412314, 265.06...","[1.8056335288815244, 460.887140727353, 188.545...","[3.930581414931163, 66.47366522118769, 162.754...",...,1.694804e+01,3.769551e+02,4.608871e+02,1.627543e+02,2.497643e+03,4.056641e+02,9.344997e+02,3.786123e+03,6.547115e+01,7.242545e+01
3,"[9.973353120030442, 28.2619963102494, 14.38229...","[3.339817761702652, 14.801599857970647, 12.212...","[0.3284044685821844, 75.5932501995836, 75.4522...","[5.08398572256648, 35.7227828002662, 45.537851...","[7.850055378117495, 53.332377188651456, 76.014...","[0.6216171727008639, 3.908380139941891, 5.0226...","[4.585471845722352, 6.558552133438308, 2.54425...","[664.625402737239, 1919.4855781264748, 957.015...","[809.4372626321455, 2556.0102889379455, 1171.6...","[14.720792807976574, 48.91865373624429, 24.490...",...,1.092489e+02,1.919486e+03,2.556010e+03,3.229554e+02,5.920535e+02,6.889802e+02,6.789933e+02,2.090250e+03,3.184835e+02,5.230859e+02
4,"[100.39960273457491, 526.1345016532301, 227.63...","[0.5219397079838253, 4.269375063764799, 13.705...","[0.4086631098849882, 46.60535901139473, 15.988...","[15.599683743996795, 90.78275377428318, 19.256...","[0.006544580049229647, 18.833619592050677, 6.6...","[0.9104917364503021, 9.550176782469505, 25.047...","[0.5493073361536257, 9.023236856585955, 23.072...","[93.11015677098604, 258.7002491500516, 175.154...","[315.8967713987784, 945.5710521890604, 575.556...","[0.06105997075121417, 4.939782349996261, 3.256...",...,5.085365e+01,2.587002e+02,9.455711e+02,1.153862e+02,1.783934e+02,2.732552e+02,9.680241e+02,7.037665e+03,1.306786e+02,1.385917e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,"[1402174.9690506302, 11325643.312826397, 33666...","[1397988.1392077552, 11326771.991035966, 33724...","[1388816.9657274298, 11190900.59836471, 333111...","[1395083.8850429433, 11347024.033110455, 33725...","[1419839.5093205227, 11389608.830857856, 33669...","[1419281.2147274155, 11375065.48050655, 336592...","[1422559.9421238878, 11404024.791714335, 33701...","[1433831.2028574906, 11471628.95011223, 339148...","[1442327.4158635808, 11529